# Python's Stack Machine Architecture: A Deep Dive

This notebook explores Python's stack machine architecture, focusing on how Python bytecode is executed. We'll use the `dis` module to inspect bytecode and understand stack behavior.

## 1. What is a Stack Machine?

- A stack machine is a type of computer architecture that uses a stack data structure to perform operations.
- Unlike register-based machines, stack machines manipulate data directly on a stack.
- **Stack:** A Last-In, First-Out (LIFO) structure.

```
+-------+  <- Top of Stack
|  Data  |
|  Data  |
|  Data  |
+-------+  <- Bottom of Stack
```

## 2. Python's Stack Machine

- CPython uses a stack machine architecture for executing bytecode.
- Python code is compiled into bytecode instructions that operate on a stack.
- The CPython interpreter processes these instructions using an evaluation loop.

## 3. Bytecode and Instructions

Examples:
- `LOAD_FAST`: Push a local variable.
- `LOAD_CONST`: Push a constant.
- `BINARY_ADD`: Pop two, add, and push result.
- `BINARY_MULTIPLY`: Pop two, multiply, and push result.
- `RETURN_VALUE`: Pop and return the top value.

Simplified Bytecode Structure:
```
+--------+--------+
| Opcode | Oparg  |
+--------+--------+
  (1 byte) (1 byte)
```

## 4. Example: `(x + 1) * y`
### 4.1. Python Code

In [ ]:
def sample_function(x, y):
    return (x + 1) * y

### 4.2. Bytecode Disassembly

In [ ]:
import dis
dis.dis(sample_function)

  1           0 RESUME                   0

  2           2 LOAD_FAST                0 (x)
              4 LOAD_CONST               1 (1)
              6 BINARY_OP                0 (+)
             10 LOAD_FAST                1 (y)
             12 BINARY_OP                5 (*)
             16 RETURN_VALUE


### 4.3. Bytecode Explanation

```
  2           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 LOAD_FAST                1 (y)
              8 BINARY_MULTIPLY
             10 RETURN_VALUE
```

- `LOAD_FAST 0 (x)`: Push `x` onto the stack.
- `LOAD_CONST 1 (1)`: Push constant `1`.
- `BINARY_ADD`: Pop `x` and `1`, add, push `x + 1`.
- `LOAD_FAST 1 (y)`: Push `y`.
- `BINARY_MULTIPLY`: Pop `(x + 1)` and `y`, multiply, push `(x + 1) * y`.
- `RETURN_VALUE`: Pop and return the result.

### 4.4. Stack Diagram Evolution
- Initially: `[]`
- After `LOAD_FAST x`: `[x]`
- After `LOAD_CONST 1`: `[x, 1]`
- After `BINARY_ADD`: `[x + 1]`
- After `LOAD_FAST y`: `[x + 1, y]`
- After `BINARY_MULTIPLY`: `[(x + 1) * y]`
- After `RETURN_VALUE`: `[]`

## 5. Deeper Dive with `dis.Bytecode`

In [ ]:
for instr in dis.Bytecode(sample_function):
    print(instr)

Instruction(opname='RESUME', opcode=151, arg=0, argval=0, argrepr='', offset=0, starts_line=1, is_jump_target=False, positions=Positions(lineno=1, end_lineno=1, col_offset=0, end_col_offset=0))
Instruction(opname='LOAD_FAST', opcode=124, arg=0, argval='x', argrepr='x', offset=2, starts_line=2, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=12, end_col_offset=13))
Instruction(opname='LOAD_CONST', opcode=100, arg=1, argval=1, argrepr='1', offset=4, starts_line=None, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=16, end_col_offset=17))
Instruction(opname='BINARY_OP', opcode=122, arg=0, argval=0, argrepr='+', offset=6, starts_line=None, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=12, end_col_offset=17))
Instruction(opname='LOAD_FAST', opcode=124, arg=1, argval='y', argrepr='y', offset=10, starts_line=None, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=21, end_col_off

## 6. Stack Effect Analysis

In [ ]:
depth = 0
max_depth = 0
for instr in dis.Bytecode(sample_function):
    effect = dis.stack_effect(instr.opcode, instr.arg)
    depth += effect
    max_depth = max(max_depth, depth)
    print(f"{instr.opname:<20} stack_effect={effect:+2} depth_now={depth:2}")
print(f"Maximum stack depth during execution: {max_depth}")

RESUME               stack_effect=+0 depth_now= 0
LOAD_FAST            stack_effect=+1 depth_now= 1
LOAD_CONST           stack_effect=+1 depth_now= 2
BINARY_OP            stack_effect=-1 depth_now= 1
LOAD_FAST            stack_effect=+1 depth_now= 2
BINARY_OP            stack_effect=-1 depth_now= 1
RETURN_VALUE         stack_effect=-1 depth_now= 0
Maximum stack depth during execution: 2


## 7. Why Understand Stack Machine Architecture?
- **Performance Tuning:** Understand cost of operations.
- **Debugging:** Helpful for reverse engineering compiled code.
- **CPython Internals:** Crucial for deep diving into Python internals.

## 8. Optimization (Simple Examples)
- **Minimize Attribute/Global Lookups:** Use locals when possible.
- **Local Variables Are Fast:** `LOAD_FAST` is quicker than `LOAD_GLOBAL`.
- **Function Call Overhead:** Inlining small functions can help sometimes.
- **List Comprehensions/Generators:** Generators save memory.

**Caveat:** Focus on readability first!

### 8.1. Example: Local vs. Global

In [ ]:
import time

GLOBAL_VAR = 10

def use_local():
    local_var = GLOBAL_VAR
    for _ in range(1000000):
        result = local_var * 2
    return result

def use_global():
    for _ in range(1000000):
        result = GLOBAL_VAR * 2
    return result

# Timing
start = time.time()
use_local()
end = time.time()
print(f"use_local: {end - start:.4f} sec")

start = time.time()
use_global()
end = time.time()
print(f"use_global: {end - start:.4f} sec")

# Disassembly
print("\nuse_local disassembly:")
dis.dis(use_local)

print("\nuse_global disassembly:")
dis.dis(use_global)

use_local: 0.0371 sec
use_global: 0.0355 sec

use_local disassembly:
  5           0 RESUME                   0

  6           2 LOAD_GLOBAL              0 (GLOBAL_VAR)
             14 STORE_FAST               0 (local_var)

  7          16 LOAD_GLOBAL              3 (NULL + range)
             28 LOAD_CONST               1 (1000000)
             30 PRECALL                  1
             34 CALL                     1
             44 GET_ITER
        >>   46 FOR_ITER                 7 (to 62)
             48 STORE_FAST               1 (_)

  8          50 LOAD_FAST                0 (local_var)
             52 LOAD_CONST               2 (2)
             54 BINARY_OP                5 (*)
             58 STORE_FAST               2 (result)
             60 JUMP_BACKWARD            8 (to 46)

  9     >>   62 LOAD_FAST                2 (result)
             64 RETURN_VALUE

use_global disassembly:
 11           0 RESUME                   0

 12           2 LOAD_GLOBAL              1 (NULL + 

### 8.3. Example: List Comprehension vs. Loop

In [1]:
import dis
import timeit
import sys

# --- List comprehension version ---
def list_comp():
    return [x * 2 for x in range(1000000)]

# --- For loop version ---
def for_loop():
    result = []
    for x in range(1000000):
        result.append(x * 2)
    return result

# --- Time each function ---
time_lc = timeit.timeit("list_comp()", globals=globals(), number=1000)
time_fl = timeit.timeit("for_loop()", globals=globals(), number=1000)

print(f"List comprehension: {time_lc:.4f} sec")
print(f"For loop:           {time_fl:.4f} sec")

# --- Memory usage ---
lc = list_comp()
fl = for_loop()

print(f"Memory (list comp): {sys.getsizeof(lc)} bytes")
print(f"Memory (for loop):  {sys.getsizeof(fl)} bytes")

# --- Disassemble to show bytecode difference ---
print("\nDisassembly: list_comp")
dis.dis(list_comp)

print("\nDisassembly: for_loop")
dis.dis(for_loop)

# -----------------------------
# 📌 Comments on performance:
#
# 1. In list_comp:
#    - Uses LIST_APPEND: a fast, specialized opcode for adding to lists.
#    - Avoids method lookups like `list.append`.
#    - Compiled into an internal code object (<listcomp>) which avoids scope pollution.
#
# 2. In for_loop:
#    - Uses LOAD_METHOD, CALL, and POP_TOP to call `append()`.
#    - These are heavier due to dynamic method resolution and stack operations.
#
# 3. Memory:
#    - Both return the same list type, but list comprehension avoids extra local references
#      and intermediate method objects, making it slightly more memory efficient.
#
# 🔍 Overall:
#    - List comprehensions are faster due to lower overhead at the bytecode level.
#    - They are also slightly more memory efficient for large transformations.
# -----------------------------


List comprehension: 63.5134 sec
For loop:           61.6090 sec
Memory (list comp): 8448728 bytes
Memory (for loop):  8448728 bytes

Disassembly: list_comp
  6           0 RESUME                   0

  7           2 LOAD_CONST               1 (<code object <listcomp> at 0x7f6061d03430, file "<ipython-input-1-ca3d9d844839>", line 7>)
              4 MAKE_FUNCTION            0
              6 LOAD_GLOBAL              1 (NULL + range)
             18 LOAD_CONST               2 (1000000)
             20 PRECALL                  1
             24 CALL                     1
             34 GET_ITER
             36 PRECALL                  0
             40 CALL                     0
             50 RETURN_VALUE

Disassembly of <code object <listcomp> at 0x7f6061d03430, file "<ipython-input-1-ca3d9d844839>", line 7>:
  7           0 RESUME                   0
              2 BUILD_LIST               0
              4 LOAD_FAST                0 (.0)
        >>    6 FOR_ITER                 7 (

## 9. Conclusion

Understanding Python’s stack machine deepens your understanding of Python internals, performance tuning, and low-level debugging. But always remember: clarity and readability come first in writing Python code.